<a href="https://colab.research.google.com/github/nishchaymisra13/Titanic-using-Random-Forest-and-XgBoost/blob/master/Titanic_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns

In [9]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (1).csv


In [0]:
import io
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [11]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test (1).csv


In [12]:
import io

test=pd.read_csv(io.BytesIO(uploaded['test.csv']))
test["Survived"]=np.nan
data=train.append(test,sort=False)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [14]:
data["Age"]=data.fillna(data["Age"].median)
data["Embarked"]=data.fillna("S")
data["Fare"]=data.fillna(data["Fare"].median)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1309 non-null object
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1309 non-null object
Cabin          295 non-null object
Embarked       1309 non-null object
dtypes: float64(1), int64(4), object(7)
memory usage: 132.9+ KB


In [15]:
data["Family Size"]= data["SibSp"]+data["Parch"]+1
data=data.drop(["Name","Cabin","Embarked"],axis=1)
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Family Size
0,1,0.0,3,male,1,1,0,A/5 21171,1,2
1,2,1.0,1,female,2,1,0,PC 17599,2,2
2,3,1.0,3,female,3,0,0,STON/O2. 3101282,3,1
3,4,1.0,1,female,4,1,0,113803,4,2
4,5,0.0,3,male,5,0,0,373450,5,1


In [16]:
data["Sex"]=pd.get_dummies(data["Sex"])
data["Ticket"]=pd.get_dummies(data["Ticket"])
data.head()


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Family Size
0,1,0.0,3,0,1,1,0,0,1,2
1,2,1.0,1,1,2,1,0,0,2,2
2,3,1.0,3,1,3,0,0,0,3,1
3,4,1.0,1,1,4,1,0,0,4,2
4,5,0.0,3,0,5,0,0,0,5,1


In [0]:
col= ["Age","Fare","Family Size","Parch","SibSp"]

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
data[col]= scaler.fit_transform(data[col])


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [19]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Family Size
0,1,0.0,3,0,0.000000,0.125,0.0,0,0.000000,0.1
1,2,1.0,1,1,0.000765,0.125,0.0,0,0.000765,0.1
2,3,1.0,3,1,0.001529,0.000,0.0,0,0.001529,0.0
3,4,1.0,1,1,0.002294,0.125,0.0,0,0.002294,0.1
4,5,0.0,3,0,0.003058,0.000,0.0,0,0.003058,0.0


In [20]:
data.isnull().sum()


PassengerId      0
Survived       418
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Family Size      0
dtype: int64

In [0]:
train_cleaned = data[data['Survived'].notnull()]
test_cleaned = data[data['Survived'].isnull()]

In [22]:
train_cleaned.shape

(891, 10)

In [23]:
test_cleaned.shape

(418, 10)

In [0]:
X= train_cleaned.drop("Survived",axis=1)
y=train_cleaned["Survived"]

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix,mean_absolute_error


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
rfc.score(X_test,y_test)

0.6871508379888268

In [26]:
from xgboost import XGBClassifier
classifier = XGBClassifier(colsample_bylevel= 0.9999999,
                    colsample_bytree = 0.9999999, 
                    gamma=0.99999,
                    max_depth= 5,
                    min_child_weight= 1,
                    n_estimators= 1000,
                    nthread= 4,
                    random_state= 2,
                    silent= True)
classifier.fit(X,y)
classifier.score(X_test,y_test)

0.8603351955307262

In [0]:
test_cleaned=test_cleaned.drop("Survived",axis=1)


In [31]:
xgb_predict = classifier.predict(test_cleaned).astype(np.uint8)
output_xgb= pd.DataFrame({ 'PassengerId' : test_cleaned["PassengerId"], 'Survived': xgb_predict })
output_xgb.to_csv('submission-xgb.csv', index=False)
output_xgb.head()


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
